In [1]:
import snap
import time, pandas as pd, pickle, json, networkx as nx, numpy as np
from networkx.readwrite import json_graph

In [2]:
cons = pd.read_csv("../REST/static/filtered_twitter_connections.csv", index_col="id")
cons.sample(5)

,from_user_id,to_user_id,formation
id,,,
1741,334139315,1063900364,{'2018.05.24': True}
4325,1110823566,2545544532,{'2018.05.24': True}
8514,835028362032742400,1110823566,{'2018.05.08': True}
6084,2568525291,2734040880,{'2018.05.08': True}
3408,630367566,294182546,{'2018.05.24': True}


In [3]:
twu_with_orgs = pd.read_csv("../REST/static/filtered_twitter_users.csv", index_col="id")
twu_with_orgs.sample(5,random_state=42)

,followers_count,friends_count,is_org,lang,match_name,match_ratio,name,screen_name,truncated_id
id,,,,,,,,,
4.158962e+09,17.0,259.0,False,tr,mustafa karan,92.0,mustafa kar,mustafa07301267,415896221
9.410563e+17,1.0,74.0,False,tr,mehmet akinci,95.0,mehmet mehmet,mehmetm88352186,941056302
7.111740e+17,180.0,1554.0,False,tr,yusuf adiguzel,96.0,Yusuf ADIGÜZEL,ysferc38,711174008
1.578003e+09,41.0,171.0,False,en,ahmed ahmed,95.0,Mikaeel Ahmed,AhmedMikaeel,157800302
1.053651e+09,56.0,149.0,False,tr,hakim alizada,92.0,Hakim Alizade,HakimAlizade,105365131


In [4]:
non_orgs = twu_with_orgs[~twu_with_orgs.is_org]
orgs = set(twu_with_orgs[twu_with_orgs.is_org].truncated_id)

In [5]:
nxg = nx.DiGraph()
truncate = lambda x: int(str(int(x))[:9])
for _, row in cons.iterrows():    
    from_ = truncate(row["from_user_id"])
    to = truncate(row["to_user_id"])
    if from_ in non_orgs.truncated_id and to in non_orgs.truncated_id:
        nxg.add_edge(from_, to)
nxg = nxg.to_directed()

In [6]:
len(nxg.nodes())

554

In [7]:
g = snap.PUNGraph.New()

In [8]:
for n in nxg.nodes():
    g.AddNode(n)

In [9]:
for f,t in nxg.edges():
    g.AddEdge(f, t)

In [10]:
CmtyV = snap.TCnComV()

## Calculating communities for nodes that had an edge at least once

In [11]:
start = time.time()
modularity = snap.CommunityGirvanNewman(g, CmtyV)
print("Took:",time.time()-start)
print ("The modularity of the network is %f" % modularity)

('Took:', 13.375367879867554)
The modularity of the network is 0.619059


In [12]:
nodes_communities = {} # {node: [community]}
for i, Cmty in enumerate(CmtyV):
    print ("Community")
    for NI in Cmty:
        nodes_communities.setdefault(NI, [])
        nodes_communities[NI].append(i+2)
        print (NI)
    print("-"*10)

Community
22726182
31348078
54890014
58895857
59087918
67326856
78625770
86759560
90942568
91697633
93257032
99132901
99529992
99947241
105561852
116874386
119125729
122483508
122663527
122762092
127227279
142105410
142638478
145593371
148740936
162000619
162417303
174239586
179632598
180698952
182798429
185153719
190175889
202960799
212853481
223989726
224170610
224265589
225548294
225925670
226497085
242772912
257963571
258835545
267598434
274019211
283868273
288988846
292138819
292878286
293464262
293910628
294182546
314054889
314777891
316015497
326711720
328485693
328899293
331175494
334509225
335134002
337061839
348415792
349249329
349791252
359872305
362783413
365925387
367753207
376124670
376196641
380937967
395971942
397429102
419545826
425411580
427576690
429101069
438540004
439156133
442653736
446303177
449856858
450064320
452056032
460049810
467503701
481423604
498656061
501589589
507737255
512309670
522595967
535239715
551235868
553444867
557654297
561229157
566067329
5685

In [13]:
set([i[0] for i in nodes_communities.values()])

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [14]:
def assign_com(node):
    try:
        return str(nodes_communities[node][0])
    except KeyError:
        if node in orgs:
            return "foci"  # this is a foci, not a user
        else:
            return "1" # there are no connections for this node
    
twu_with_orgs["community"] = twu_with_orgs.truncated_id.apply(lambda node: assign_com(node))
twu_with_orgs.sample(5, random_state=42)

,followers_count,friends_count,is_org,lang,match_name,match_ratio,name,screen_name,truncated_id,community
id,,,,,,,,,,
4.158962e+09,17.0,259.0,False,tr,mustafa karan,92.0,mustafa kar,mustafa07301267,415896221,1
9.410563e+17,1.0,74.0,False,tr,mehmet akinci,95.0,mehmet mehmet,mehmetm88352186,941056302,1
7.111740e+17,180.0,1554.0,False,tr,yusuf adiguzel,96.0,Yusuf ADIGÜZEL,ysferc38,711174008,1
1.578003e+09,41.0,171.0,False,en,ahmed ahmed,95.0,Mikaeel Ahmed,AhmedMikaeel,157800302,1
1.053651e+09,56.0,149.0,False,tr,hakim alizada,92.0,Hakim Alizade,HakimAlizade,105365131,1


In [15]:
twu_with_orgs.to_csv("../datasets/filtered_twitter_users.csv", index_label="id")
twu_with_orgs.to_csv("../REST/static/filtered_twitter_users.csv", index_label="id")

## Calculating network diameter

In [16]:
with open('../REST/static/networks/twitter_users_graph2.json', 'r') as f:
    data = json.load(f)
    nxg = json_graph.node_link_graph(data, directed=True)
nxg = nxg.to_directed()

In [17]:
def networkx_to_snappy(nxg, directed=False):
    if directed:
        g = snap.PNGraph.New()
    else:
        g = snap.PUNGraph.New()
        
    for n in nxg.nodes():
        g.AddNode(n)
    for f,t in nxg.edges():
        g.AddEdge(f, t)
        
    return g

In [18]:
for node in nxg.nodes():
    nxg.nodes[node]["community"] = twu_with_orgs.loc[node]["community"]

In [19]:
data = nx.node_link_data(nxg)
data["modularity"] = modularity
data["number_of_communities"] = len(CmtyV)

snappy_directed = networkx_to_snappy(nxg, True)
snappy_undirected = networkx_to_snappy(nxg, False)

diam_directed = snap.GetBfsFullDiam(snappy_directed, 100, True)
diam_undirected = snap.GetBfsFullDiam(snappy_undirected, 100, False)

data["diameter_directed"]=diam_directed
data["diameter_undirected"]=diam_undirected
with open('../visualization/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)
    
with open('../REST/static/networks/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)

In [20]:
len(nxg.nodes())

722

In [21]:
len(nxg.edges())

1280